This notebook is centered around trying to figure out how to concurrently handle animating the LEDs while listening for events or some way to interrupt the animation. Ideally, it would be nice to be to have some sort of system that:

- Manages the animation in a separate process, and
- Has a process that listen for events to take action the first process

You can imagine that:

Some program sends message to -> Some animation management process which controls -> Animation process

This experiment will focus on the last two. The idea is to use some of the facilities afforded to us in the `multiprocessing` package to do this.

## Approach

Because of the GIL, we do not have true parallel multithreading. Since we want the thing that communicates to the LEDs to run at the same time as the thing listening for events, we are forced to use multiprocessing.

We can have a process (we'll call it the animation manager process for now) that is responsible for two things:

- listening for messages; we'll pretend these messages contain animations
- killing the current animation process and starting a new one with the new animation

For listening for messages, we'll use the `multiprocessing.connection` package. For spawning a worker to play our animation, we'll use the `multiprocessing.Process` class.

In [ ]:
from pixelbuf_pi_animation.data import *
from pixelbuf_pi_animation.player import SimplePixelBufPlayer

import adafruit_dotstar
import board


num_leds = 11
pixels = adafruit_dotstar.DotStar(board.SCLK, board.MOSI, num_leds, pixel_order=adafruit_dotstar.GBR, auto_write=False)

In [ ]:
from multiprocessing import Process
from multiprocessing.connection import Connection, Listener


class AnimationManager:
    
    def __init__(self, player: SimplePixelBufPlayer, port: int, auth_key: bytes):
        self._player = player
        self._listener = Listener(address=('localhost', port), authkey=auth_key)
        self._process = None
    
    @staticmethod
    def render_animation(player: SimplePixelBufPlayer, animation: Animation):
        """
        This is our animation "worker" and handles the communication with the LEDs
        """
        player.load(animation)
        player.play()
    
    def start(self):
        """
        Main loop
        """
        while True:
            try:
                # Accept the connection and get our message
                connection = self._listener.accept()
                animation_message = connection.recv()
                
                # Kill the current animation
                if self._process and self._process.is_alive():
                    self._process.terminate()
                    
                # Start a new process
                self._process = Process(target=AnimationManager.render_animation, args=(self._player, animation_message))
                self._process.start()
                
                # Send a response back to the client then close the connection
                connection.send(self._process.pid)
                connection.close()
                
            except Exception as e:
                # If we have a connection, let them know we had an issue
                if connection:
                    connection.send(-1)
                    connection.close()
                print(e)


In [ ]:
# This cell will run infinitely

player = SimplePixelBufPlayer(pixels)
animation_manager = AnimationManager(player=player, port=10101, auth_key=b'supersecretkey')
animation_manager.start()